In [14]:
import pandas as pd
import yaml
root_path = '/local/work/merengelke/aipal/'
config = yaml.load(open('cfg.yaml', 'r'), Loader=yaml.FullLoader)

cities_countries = config['cities_countries']
paths = [f"{root_path}{city_country}/aipal/predict.csv" for city_country in cities_countries]

df = pd.DataFrame()
for path in paths:
    df_small = pd.read_csv(path)
    df_small['city_country'] = path.split('/')[-3]
    df = pd.concat([df, df_small])
    
# age over 18
if config['is_adult']:
    df = df[df['age'] > 18]
else:
    df = df[df['age'] <= 18]
df.drop(columns=['ELN', 'Diagnosis', 'additional.diagnosis.details..lineage.etc', 'lineage.details'], inplace=True)

In [15]:
df.columns


Index(['ID', 'age', 'class', 'MCV_fL', 'PT_percent', 'LDH_UI_L', 'MCHC_g_L',
       'WBC_G_L', 'Fibrinogen_g_L', 'Monocytes_G_L', 'Platelets_G_L',
       'Lymphocytes_G_L', 'sex', 'Monocytes_percent', 'prediction.ALL',
       'prediction.AML', 'prediction.APL', 'city_country', 'UPN'],
      dtype='object')

In [16]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import jensenshannon

# Load your data (assuming a pandas DataFrame `df`)
# Ensure that the following columns are in your dataframe: 'prediction.ALL', 'prediction.AML', 'prediction.APL'

# Extract prediction columns
predictions = df[['prediction.ALL', 'prediction.AML', 'prediction.APL']]

# Normalize the predictions (if not already normalized)
predictions = predictions.div(predictions.sum(axis=1), axis=0)

# Calculate pairwise Jensen-Shannon divergence
js_divergences = []
for i in range(len(predictions)):
    # You can compare distributions across each row
    dist = predictions.iloc[i]
    
    # Compute Jensen-Shannon divergence between distributions
    js_div = jensenshannon(dist, dist, base=2)  # Use base 2 for Shannon's entropy
    js_divergences.append(js_div)

# Store results in a new column
df['JS_Divergence'] = js_divergences

# Display results
print(df.head())  # Alternatively, you can save the DataFrame to a file with df.to_csv('output.csv')

# Now df contains the Jensen-Shannon divergences for each prediction set

     ID   age class  MCV_fL  PT_percent  LDH_UI_L  MCHC_g_L  WBC_G_L  \
1  MHH2  24.0   AML    85.0        56.0     183.0      32.0     18.2   
2  MHH3  30.0   ALL    83.0        70.0    2018.0      33.0     59.0   
3  MHH4  40.0   AML   108.0       116.0     173.0      35.0      1.0   
4  MHH5  49.0   AML    91.0        62.7     188.0      37.0      9.9   
5  MHH6  46.0   AML    81.0        62.4    1173.0      33.0     42.0   

   Fibrinogen_g_L  Monocytes_G_L  Platelets_G_L  Lymphocytes_G_L     sex  \
1           436.0           1.27          311.0             2.55  Female   
2             NaN           0.00           51.0             2.95    Male   
3           425.0           0.03           76.0             0.93  Female   
4           700.0           5.77           25.0             1.88    Male   
5           700.0           1.26           86.0             0.42  Female   

   Monocytes_percent  prediction.ALL  prediction.AML  prediction.APL  \
1           6.978022        0.374188  